# Loading an LLM

In [ ]:
!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

# If this command does not work for you, you can use the link directly to download the model
# https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

In [46]:
!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

--2024-12-20 17:33:26--  https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf
Resolving huggingface.co (huggingface.co)... 18.238.176.129, 18.238.176.103, 18.238.176.8, ...
Connecting to huggingface.co (huggingface.co)|18.238.176.129|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/41/c8/41c860f65b01de5dc4c68b00d84cead799d3e7c48e38ee749f4c6057776e2e9e/5d99003e395775659b0dde3f941d88ff378b2837a8dc3a2ea94222ab1420fad3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Phi-3-mini-4k-instruct-fp16.gguf%3B+filename%3D%22Phi-3-mini-4k-instruct-fp16.gguf%22%3B&Expires=1734975206&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczNDk3NTIwNn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzQxL2M4LzQxYzg2MGY2NWIwMWRlNWRjNGM2OGIwMGQ4NGNlYWQ3OTlkM2U3YzQ4ZTM4ZWU3NDlmNGM2MDU3Nzc2ZTJlOWUvNWQ5OTAwM2UzOTU3NzU2NTliMGRkZTNmOTQ

In [47]:
!pip install langchain-community

In [48]:
!pip install llama-cpp-python

In [49]:
from langchain import LlamaCpp

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="Phi-3-mini-4k-instruct-fp16.gguf",
    n_gpu_layers=-1,
    max_tokens=500,
    n_ctx=2048,
    seed=42,
    verbose=False
)

llama_new_context_with_model: n_batch is less than GGML_KQ_MASK_PAD - increasing to 32
llama_new_context_with_model: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized


In [50]:
llm.invoke("Hi! My name is Emmanuel. What is 1 + 1?")

''

### Chains

In [51]:
from langchain import PromptTemplate

# Create a prompt template with the "input_prompt" variable
template = """<s><|user|>
{input_prompt}<|end|>
<|assistant|>"""
prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt"]
)

In [52]:
basic_chain = prompt | llm

In [53]:
# Use the chain
basic_chain.invoke(
    {
        "input_prompt": "Hi! My name is Emmanuel. What is 0 + 1?",
    }
)

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1237: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


' Hello Emmanuel! The answer to 0 + 1 is 1.'

### Multiple Chains

In [54]:
from langchain import LLMChain

# Create a chain for the title of our story
template = """<s><|user|>
Create a title for a story about {summary}. Only return the title.<|end|>
<|assistant|>"""
title_prompt = PromptTemplate(template=template, input_variables=["summary"])
title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")

In [55]:
title.invoke({"summary": "a girl that lost her mother"})

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1237: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'summary': 'a girl that lost her mother',
 'title': ' "Whispers of a Farewell: The Journey Through Loss"'}

In [56]:
# Create a chain for the character description using the summary and title
template = """<s><|user|>
Describe the main character of a story about {summary} with the title {title}. Use only two sentences.<|end|>
<|assistant|>"""
character_prompt = PromptTemplate(
    template=template, input_variables=["summary", "title"]
)
character = LLMChain(llm=llm, prompt=character_prompt, output_key="character")

In [57]:
# Create a chain for the story using the summary, title, and character description
template = """<s><|user|>
Create a story about {summary} with the title {title}. The main charachter is: {character}. Only return the story and it cannot be longer than one paragraph<|end|>
<|assistant|>"""
story_prompt = PromptTemplate(
    template=template, input_variables=["summary", "title", "character"]
)
story = LLMChain(llm=llm, prompt=story_prompt, output_key="story")

In [58]:
# Combine all three components to create the full chain
llm_chain = title | character | story

In [59]:
llm_chain.invoke("a girl that lost her mother")

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1237: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1237: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1237: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'summary': 'a girl that lost her mother',
 'title': ' "Lily\'s Lament: A Journey Through Grief"',
 'character': " Lily, a resilient and compassionate teenager with fiery red hair and piercing green eyes, struggles to cope with the sudden loss of her beloved mother. As she embarks on an emotional journey through grief, she discovers strength in vulnerability, finding solace in newfound friendships and personal growth along the way.\n\nLily's heart-wrenching story is that of a sensitive soul who has been shaped by her loss but remains determined to keep her mother's memory alive while learning to face life's challenges with courage and grace.",
 'story': " Lily's Lament: A Journey Through Grief\n\nIn the wake of her mother's untimely departure, a fiery-haired teenager named Lily found herself adrift in an ocean of sorrow. With piercing green eyes that once shimmered with laughter and love, she grappled to understand why life had cruelly snatched away the woman who held her heart so tigh

# Memory

In [60]:
# Let's give the LLM our name
basic_chain.invoke({"input_prompt": "Hi! My name is Emmanuel. What is 0 + 1?"})

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1237: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


' Hello Emmanuel! The answer to 0 + 1 is 1.'

In [61]:
# Next, we ask the LLM to reproduce the name
basic_chain.invoke({"input_prompt": "What is my name?"})

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1237: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


" I'm unable to determine your name as I don't have access to personal data of individuals. If you need assistance with something specific, feel free to ask!"

## ConversationBuffer

In [62]:
# Create an updated prompt template to include a chat history
template = """<s><|user|>Current conversation:{chat_history}

{input_prompt}<|end|>
<|assistant|>"""

prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt", "chat_history"]
)

In [63]:
from langchain.memory import ConversationBufferMemory

# Define the type of Memory we will use
memory = ConversationBufferMemory(memory_key="chat_history")

# Chain the LLM, Prompt, and Memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

In [66]:
# Generate a conversation and ask a basic question
llm_chain.invoke({"input_prompt": "Hi! My name is Emmanuel. What is 0 + 1?"})

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1237: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'Hi! My name is Emmanuel. What is 0 + 1?',
 'chat_history': "Human: Hi! My name is Maarten. What is 1 + 1?\nAI:  The answer to 1 + 1 is 2. It's a basic arithmetic operation where you add one unit to another, resulting in two units in total.\n\nHere is the calculation:\n\n1 + 1 = 2\nHuman: What is my name?\nAI:  Your name is Maarten.\n\nI'm an AI and I don't have a personal name, but you can call me Assistant.",
 'text': " The answer to 0 + 1 is 1. In this basic arithmetic operation, when you add one unit to zero units, the result is one unit in total.\n\nHere's the calculation:\n\n0 + 1 = 1"}

In [68]:
# Does the LLM remember the name we gave it?
llm_chain.invoke({"input_prompt": "What is my name?"})

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1237: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What is my name?',
 'chat_history': "Human: Hi! My name is Maarten. What is 1 + 1?\nAI:  The answer to 1 + 1 is 2. It's a basic arithmetic operation where you add one unit to another, resulting in two units in total.\n\nHere is the calculation:\n\n1 + 1 = 2\nHuman: What is my name?\nAI:  Your name is Maarten.\n\nI'm an AI and I don't have a personal name, but you can call me Assistant.\nHuman: Hi! My name is Emmanuel. What is 0 + 1?\nAI:  The answer to 0 + 1 is 1. In this basic arithmetic operation, when you add one unit to zero units, the result is one unit in total.\n\nHere's the calculation:\n\n0 + 1 = 1\nHuman: What is my name?\nAI:  As an AI, I don't have a personal name either, but you can refer to me as Assistant. It's great to meet you again, Emmanuel! Your question about 0 + 1 is correctly answered with the result of 1. If there's anything else you'd like to know or discuss, feel free to ask!",
 'text': " Your name is Emmanuel. I'm an AI and don't have a pers

In [ ]:
#The llm is able to remeber my name because i am saving my chat history for it to access

## ConversationBufferMemoryWindow

In [69]:
from langchain.memory import ConversationBufferWindowMemory

# Retain only the last 2 conversations in memory
memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")

# Chain the LLM, Prompt, and Memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

In [70]:
# Ask two questions and generate two conversations in its memory
llm_chain.invoke({"input_prompt":"Hi! My name is Emmanuel and I am 25 years old. What is 1 + 1?"})
llm_chain.invoke({"input_prompt":"What is 0 + 3?"})

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1237: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1237: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What is 0 + 3?',
 'chat_history': "Human: Hi! My name is Emmanuel and I am 25 years old. What is 1 + 1?\nAI:  The sum of 1 + 1 is 2. This mathematical operation falls within basic arithmetic, and regardless of the personal details shared (in this case, your name, age), it remains a straightforward calculation.\n\nHowever, you might be looking for an opportunity to engage more deeply or explore other topics related to mathematics beyond simple calculations if that's something you're interested in discussing further!",
 'text': " The sum of 0 + 3 is 3. This also falls within basic arithmetic, and like before, the calculation remains straightforward regardless of any additional personal details shared.\n\nIf you're interested in discussing more advanced mathematical concepts or topics, feel free to ask!"}

In [71]:
# Check whether it knows the name we gave it
llm_chain.invoke({"input_prompt":"What is my name?"})

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1237: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What is my name?',
 'chat_history': "Human: Hi! My name is Emmanuel and I am 25 years old. What is 1 + 1?\nAI:  The sum of 1 + 1 is 2. This mathematical operation falls within basic arithmetic, and regardless of the personal details shared (in this case, your name, age), it remains a straightforward calculation.\n\nHowever, you might be looking for an opportunity to engage more deeply or explore other topics related to mathematics beyond simple calculations if that's something you're interested in discussing further!\nHuman: What is 0 + 3?\nAI:  The sum of 0 + 3 is 3. This also falls within basic arithmetic, and like before, the calculation remains straightforward regardless of any additional personal details shared.\n\nIf you're interested in discussing more advanced mathematical concepts or topics, feel free to ask!",
 'text': " Your name is Emmanuel. In response to your question, the sum of 0 + 3 is indeed 3. While we've been focusing on basic arithmetic here, if y

## ConversationSummary

In [74]:
# Create a summary prompt template
summary_prompt_template = """<s><|user|>Summarize the conversations and update with the new lines.

Current summary:
{summary}

new lines of conversation:
{new_lines}

New summary:<|end|>
<|assistant|>"""
summary_prompt = PromptTemplate(
    input_variables=["new_lines", "summary"],
    template=summary_prompt_template
)

In [75]:
from langchain.memory import ConversationSummaryMemory

# Define the type of memory we will use
memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="chat_history",
    prompt=summary_prompt
)

# Chain the LLM, prompt, and memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

In [82]:
# Generate a conversation and ask for the name
llm_chain.invoke({"input_prompt": "Hi! My name is Emmanuel. What is 0 + 1?"})
llm_chain.invoke({"input_prompt": "What is my name?"})

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1237: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1237: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1237: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1237: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What is my name?',
 'chat_history': " The AI confirmed Emmanuel's math question about the sum of 0 and 1, stating it equals 1. It reiterated its adherence to privacy policies, not having access to personal information unless mentioned in the conversation for contextual purposes only. Upon learning Emmanuel's name during their interaction, the AI acknowledged his identity while clarifying that no other personal details were known without prior mention. The AI encouraged further queries and assistance with math or any other topics as needed.",
 'text': ' Your name was not mentioned in our conversation. How can I assist you further?'}

In [81]:
# Check whether it has summarized everything thus far
llm_chain.invoke({"input_prompt": "What was the first question I asked?"})

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1237: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1237: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What was the first question I asked?',
 'chat_history': " The AI confirmed Emmanuel's math question, stating that 0 + 1 equals 1. It reiterated its privacy policy by emphasizing not knowing personal details unless mentioned within the current conversation for information retention purposes. The AI offered further assistance with math or other queries as needed and clarified it does not know the user's name in this specific context without prior mention.",
 'text': ' You first asked a math question regarding the sum of 0 and 1.'}

In [83]:
# Check what the summary is thus far
memory.load_memory_variables({})

{'chat_history': " The AI addressed a math question regarding the sum of 0 and 1, affirming it equals 1 while emphasizing privacy policies. Upon identifying Emmanuel's name as part of their interaction, the AI clarified that no further personal details were known without prior mention, and encouraged more inquiries on various topics. When asked about his own name during a new conversation, the AI reminded him it wasn't disclosed earlier and offered additional assistance if needed."}